In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=dedf0049bb81109bd890688691feb3288ea9765589d81796237d955123b93642
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import os
from google.colab import drive
from pyspark.sql import functions as F
#from pyspark.sql.functions import explode, col, concat, lit,size, desc, asc, asc_nulls_last, substring,udf,year

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
df=spark.read.csv('gdrive/My Drive/Colab Notebooks/brewery_data_complete_extended.csv',inferSchema=True,header=True)

In [ ]:
df.show(n=1, truncate=False, vertical=True)

-RECORD 0-------------------------------------------
 Batch_ID                     | 7870796             
 Brew_Date                    | 2020-01-01 00:00:19 
 Beer_Style                   | Wheat Beer          
 SKU                          | Kegs                
 Location                     | Whitefield          
 Fermentation_Time            | 16                  
 Temperature                  | 24.204250857069873  
 pH_Level                     | 5.2898454476095615  
 Gravity                      | 1.0395041267301979  
 Alcohol_Content              | 5.370842159553436   
 Bitterness                   | 20                  
 Color                        | 5                   
 Ingredient_Ratio             | 1:0.32:0.16         
 Volume_Produced              | 4666                
 Total_Sales                  | 2664.7593448382822  
 Quality_Score                | 8.57701633109399    
 Brewhouse_Efficiency         | 89.19588216376087   
 Loss_During_Brewing          | 4.104987659187

In [ ]:
df.printSchema()

root
 |-- Batch_ID: integer (nullable = true)
 |-- Brew_Date: timestamp (nullable = true)
 |-- Beer_Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Fermentation_Time: integer (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- pH_Level: double (nullable = true)
 |-- Gravity: double (nullable = true)
 |-- Alcohol_Content: double (nullable = true)
 |-- Bitterness: integer (nullable = true)
 |-- Color: integer (nullable = true)
 |-- Ingredient_Ratio: string (nullable = true)
 |-- Volume_Produced: integer (nullable = true)
 |-- Total_Sales: double (nullable = true)
 |-- Quality_Score: double (nullable = true)
 |-- Brewhouse_Efficiency: double (nullable = true)
 |-- Loss_During_Brewing: double (nullable = true)
 |-- Loss_During_Fermentation: double (nullable = true)
 |-- Loss_During_Bottling_Kegging: double (nullable = true)



In [ ]:
df_desc=df.describe(df.columns)

In [ ]:
df_pd=df_desc.toPandas()

In [ ]:
df_pd=df_pd.T

In [ ]:
df_pd.columns = df_pd.iloc[0]

In [ ]:
df_pd.iloc[5:,:]

summary,count,mean,stddev,min,max
Fermentation_Time,10000000,14.500898,2.872006096518228,10,19
Temperature,10000000,19.999898511018827,2.8870297120328576,15.000001163771435,24.999998289887966
pH_Level,10000000,4.999940543893489,0.28863762894103545,4.500000005935603,5.499999818305633
Gravity,10000000,1.0550028700788692,0.014434649211836703,1.0300000027891478,1.0799999980323736
Alcohol_Content,10000000,5.249709006579308,0.43296144791729213,4.500000235642255,5.999999932506248
Bitterness,10000000,39.4961996,11.545572488490313,20,59
Color,10000000,11.9993459,4.321170228005893,5,19
Ingredient_Ratio,10000000,None,None,1:0.20:0.10,1:0.50:0.30
Volume_Produced,10000000,2749.0309594,1299.078133259011,500,4999
Total_Sales,10000000,10497.785343940232,5485.995544804044,1000.0009630942443,19999.99964105241


In [ ]:
df.groupBy('Beer_Style')\
.sum("Volume_Produced")\
.withColumnRenamed("sum(Volume_Produced)", "Volume_Produced_Sum")\
.sort("Beer_Style").show()

+----------+-------------------+
|Beer_Style|Volume_Produced_Sum|
+----------+-------------------+
|       Ale|         3438491666|
|       IPA|         3437465053|
|     Lager|         3437866881|
|   Pilsner|         3435554213|
|    Porter|         3437816327|
|      Sour|         3432995872|
|     Stout|         3436714200|
|Wheat Beer|         3433405382|
+----------+-------------------+



In [ ]:
df.groupBy('SKU')\
.sum("Volume_Produced")\
.withColumnRenamed("sum(Volume_Produced)", "Volume_Produced_Sum")\
.sort("SKU").show()

+-------+-------------------+
|    SKU|Volume_Produced_Sum|
+-------+-------------------+
|Bottles|         6876364027|
|   Cans|         6870005491|
|   Kegs|         6870118905|
|  Pints|         6873821171|
+-------+-------------------+



In [ ]:
df.groupBy('Location')\
.sum("Volume_Produced")\
.withColumnRenamed("sum(Volume_Produced)", "Volume_Produced_Sum")\
.sort("Location").show()

+---------------+-------------------+
|       Location|Volume_Produced_Sum|
+---------------+-------------------+
|Electronic City|         2744848424|
|     HSR Layout|         2750592166|
|    Indiranagar|         2749066239|
|      Jayanagar|         2750760812|
|    Koramangala|         2749833908|
|    Malleswaram|         2747512261|
|   Marathahalli|         2748916542|
|    Rajajinagar|         2749725468|
|     Whitefield|         2748458725|
|      Yelahanka|         2750595049|
+---------------+-------------------+



In [ ]:
df.select(['Location','Brew_Date','Volume_Produced'])\
.withColumn('Brew_Month', F.date_format('Brew_Date', "yyyy-MM")).show()

+---------------+-------------------+---------------+----------+
|       Location|          Brew_Date|Volume_Produced|Brew_Month|
+---------------+-------------------+---------------+----------+
|     Whitefield|2020-01-01 00:00:19|           4666|   2020-01|
|     Whitefield|2020-01-01 00:00:31|            832|   2020-01|
|    Malleswaram|2020-01-01 00:00:40|           2115|   2020-01|
|    Rajajinagar|2020-01-01 00:01:37|           3173|   2020-01|
|   Marathahalli|2020-01-01 00:01:43|           4449|   2020-01|
|     Whitefield|2020-01-01 00:01:48|           3752|   2020-01|
|Electronic City|2020-01-01 00:01:49|            593|   2020-01|
|    Indiranagar|2020-01-01 00:01:51|           4949|   2020-01|
|Electronic City|2020-01-01 00:02:16|           2451|   2020-01|
|Electronic City|2020-01-01 00:02:32|           4305|   2020-01|
|Electronic City|2020-01-01 00:02:36|           4959|   2020-01|
|Electronic City|2020-01-01 00:02:55|           4894|   2020-01|
|    Rajajinagar|2020-01-

In [ ]:
df.select(['Location','Brew_Date','Volume_Produced'])\
.withColumn('Brew_Month', F.date_format('Brew_Date', "yyyy-MM"))\
.groupBy('Brew_Month')\
.pivot('Location')\
.sum('Volume_Produced')\
.sort('Brew_Month')\
.show()

+----------+---------------+----------+-----------+---------+-----------+-----------+------------+-----------+----------+---------+
|Brew_Month|Electronic City|HSR Layout|Indiranagar|Jayanagar|Koramangala|Malleswaram|Marathahalli|Rajajinagar|Whitefield|Yelahanka|
+----------+---------------+----------+-----------+---------+-----------+-----------+------------+-----------+----------+---------+
|   2020-01|       58157063|  57791779|   58476841| 59240364|   58033941|   58244295|    57765712|   58559281|  59268304| 58472454|
|   2020-02|       54036768|  53959900|   54539968| 54237101|   54791788|   54492559|    54298376|   54987598|  54299030| 55047953|
|   2020-03|       57938008|  58488866|   58617914| 58510168|   57998335|   58202733|    58370414|   58290870|  58434380| 58100744|
|   2020-04|       57240763|  56297919|   56936309| 57233419|   56560284|   56095060|    56291880|   55730412|  56683627| 56855324|
|   2020-05|       58597015|  58368536|   58236599| 58929325|   58173249|   

In [ ]:
df.groupBy('Beer_Style')\
.sum("Total_Sales")\
.withColumnRenamed("sum(Total_Sales)", "Total_Sales_Sum")\
.sort("Beer_Style").show()

+----------+--------------------+
|Beer_Style|     Total_Sales_Sum|
+----------+--------------------+
|       Ale|1.314638271068101...|
|       IPA|1.311356920476101...|
|     Lager|1.311766538654772...|
|   Pilsner|1.312235003270339...|
|    Porter|1.313051202605402...|
|      Sour|1.312554186649535...|
|     Stout|1.311502583634514E10|
|Wheat Beer|1.310680637581503...|
+----------+--------------------+



In [ ]:
df.groupBy('Beer_Style')\
.sum("Total_Sales")\
.withColumnRenamed("sum(Total_Sales)", "Total_Sales_Sum")\
.sort("Beer_Style").collect()[0][1]

13146382710.681017

In [ ]:
df.groupBy("Beer_Style","Location")\
.agg(F.round(F.avg("Loss_During_Brewing"),3),\
     F.round(F.avg("Loss_During_Fermentation"),3),\
     F.round(F.avg("Loss_During_Bottling_Kegging"),3))\
.sort("Beer_Style").show()

+----------+---------------+----------------------------------+---------------------------------------+-------------------------------------------+
|Beer_Style|       Location|round(avg(Loss_During_Brewing), 3)|round(avg(Loss_During_Fermentation), 3)|round(avg(Loss_During_Bottling_Kegging), 3)|
+----------+---------------+----------------------------------+---------------------------------------+-------------------------------------------+
|       Ale|   Marathahalli|                             2.991|                                  2.999|                                      3.005|
|       Ale|Electronic City|                             3.005|                                    3.0|                                      2.996|
|       Ale|    Rajajinagar|                             2.999|                                  2.999|                                      3.001|
|       Ale|      Jayanagar|                             3.005|                                    3.0|         

In [6]:
df.groupBy("Beer_Style")\
.agg(F.round(F.avg("Alcohol_Content"),3),\
     F.round(F.avg("Bitterness"),3),\
     F.round(F.avg("Color"),3))\
.sort("Beer_Style").show()

+----------+------------------------------+-------------------------+--------------------+
|Beer_Style|round(avg(Alcohol_Content), 3)|round(avg(Bitterness), 3)|round(avg(Color), 3)|
+----------+------------------------------+-------------------------+--------------------+
|       Ale|                         5.249|                   39.498|              11.999|
|       IPA|                          5.25|                   39.495|                12.0|
|     Lager|                          5.25|                   39.491|              12.001|
|   Pilsner|                          5.25|                   39.499|              11.995|
|    Porter|                          5.25|                   39.509|              11.995|
|      Sour|                         5.249|                   39.487|              12.005|
|     Stout|                          5.25|                   39.491|              12.003|
|Wheat Beer|                          5.25|                     39.5|              11.998|